# Scenario 6: Automation - Proofpoint TAP Alerts
### Description:
An organization is monitoring its email security and leveraging Proofpoint Targeted Attack Protection (TAP). Proofpoint TAP detects several email-based threats, including phishing attempts and malicious attachments, that were flagged as high confidence indicators of compromise (IOCs). The organization queries the TAP API to retrieve recent alerts and decides to share these threat indicators with a MISP instance to collaborate with other organizations and improve the overall threat intelligence landscape.

In this scenario, we will simulate the process of querying Proofpoint TAP for alerts and creating MISP events to share relevant indicators. The intelligence shared will include malicious email addresses, URLs, file hashes, and sender IP addresses associated with these threats.

### Indicators:
We will be referencing ```misp-training/scenario_notebooks/data/proofpoint_tap.json``` for specific indicators to use for this scenario. 

# Initialize environment
This section initializes the playbook environment and loads the required Python libraries. The credentials for MISP (authkey) is loaded from the file `keys.py` in the directory **vault**. A PyMISP object is created to interact with MISP and the active MISP server is displayed. By printing out the server name you know that it's possible to connect to MISP. In case of a problem, PyMISP will indicate the error with `PyMISPError: Unable to connect to MISP`.

The contents of the `keys.py` file should contain at least:
```
misp_url = "<MISP URL>"             # The URL to our MISP server
misp_key = "<MISP Authkey>"         # The MISP authkey
misp_verifycert = <True or False>   # Indicate if PyMISP should attempt to verify the certificate or ignore errors
```

It will also load the ```data/proofpoint_tap.json``` file into the variable ```tap_alerts``` to emulate a TAP API response. 

In [ ]:
from vault.keys import *

import json
from pymisp import PyMISP, MISPEvent, MISPObject


if not misp_verifycert:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("The \033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from the keys file.")

# Create the PyMISP object
misp = PyMISP(misp_url, misp_key, misp_verifycert)
print("I will use the MISP server \033[92m{}\033[90m for this playbook.\n\n".format(misp_url))

# read proofpoint_tap.json into a variable to emulate API response
with open('data/proofpoint_tap.json', 'r') as f:
    tap_alerts = json.loads(f.read())

# Iterate over TAP Alerts to Create MISP Events
This cell will iterate over all of the TAP alerts and create separate MISP Events based on the threat types. MISP Objects will be used to group alerts with the same threat type into a singular Event.

In [ ]:
from datetime import datetime

for threat_type in tap_alerts.keys():
    info = f'Proofpoint TAP {threat_type} Alerts for {datetime.utcnow().strftime("%D")}'
    if threat_type == "queryEndTime":
        continue
    elif threat_type == "clicksPermitted":
        clicks_permitted_event = MISPEvent()
        clicks_permitted_event.info = info
        clicks_permitted_event.threat_level_id = 1  # high
        clicks_permitted_event.analysis = 2  # completed
        clicks_permitted_event.distribution = 1  # this community only

        for alert in tap_alerts[threat_type]:
            sender = alert['sender']
            sender_ip = alert['senderIP']
            clicked_url = alert['url']
            classification = alert['classification']

            # double check object and relations
            cp_object = MISPObject('spearphishing-link')
            cp_object.add_attribute(object_relation="email-sender", simple_value=sender)
            cp_object.add_attribute(object_relation="sender-ip", simple_value=sender_ip)
            cp_object.add_attribute(object_relation="embedded-link", simple_value=clicked_url)
            cp_object.comment = classification

            clicks_permitted_event.add_object(cp_object)

        misp.add_event(clicks_permitted_event)

    elif threat_type == "messagesDelivered":
        for alert in tap_alerts[threat_type]:
            messages_delivered_event = MISPEvent()
            messages_delivered_event.info = info
            messages_delivered_event.threat_level_id = 2  # medium
            messages_delivered_event.analysis = 2  # completed
            messages_delivered_event.distribution = 1  # this community only

            sender = alert['sender']
            sender_header_from = alert['headerFrom']
            sender_header_reply_to = alert['headerReplyTo']
            sender_ip = alert['senderIP']
            subject = alert['subject']
            message_time = alert['messageTime']
            imposter_score = alert.get('imposterScore', 0)
            malware_score = alert.get('malwareScore', 0)
            phish_score = alert.get('phishScore', 0)
            spam_score = alert.get('spamScore', 0)

            messages_delivered_event.add_attribute(type="email-src", value=sender)
            messages_delivered_event.add_attribute(type="ip-src", value=sender_ip, category="Payload delivery")
            messages_delivered_event.add_attribute(type="email-subject", value=subject)

            if sender_header_from:
                messages_delivered_event.add_attribute(type="email-src-display-name", value=sender_header_from)
            if sender_header_reply_to:
                messages_delivered_event.add_attribute(type="email-reply-to", value=sender_header_reply_to)
            messages_delivered_event.add_attribute(type="datetime", value=message_time)

            if imposter_score > 0:
                messages_delivered_event.add_attribute(type="comment", value=imposter_score, comment="imposter score")
            if malware_score > 0:
                messages_delivered_event.add_attribute(type="comment", value=malware_score, comment="malware score")
            if phish_score > 0:
                messages_delivered_event.add_attribute(type="comment", value=phish_score, comment="phish score")
            if spam_score > 0:
                messages_delivered_event.add_attribute(type="comment", value=spam_score, comment="spam score")

            for threat in alert['threatsInfoMap']:
                classification = threat['classification']
                email_threat = threat['threat']
                threat_type = threat['threatType']

                if threat_type == "URL":
                    messages_delivered_event.add_attribute(type="domain", value=email_threat,
                                                           category="Payload delivery", comment=classification)

                elif threat_type == "ATTACHMENT":
                    messages_delivered_event.add_attribute(type="sha256", value=email_threat,
                                                           categroy="Payload delivery", comment=classification)

            for messagepart in alert['messageParts']:
                if messagepart['disposition'] == "attached":
                    attachment_filename = messagepart['filename']
                    attachment_md5 = messagepart['md5']
                    attachment_sha256 = messagepart['sha256']
                    attachment_content_type = messagepart['oContentType']

                    spearphishing_attachment_obj = MISPObject('file')
                    spearphishing_attachment_obj.comment = "malicious email attachment"
                    spearphishing_attachment_obj.add_attribute(object_relation="filename",
                                                               simple_value=attachment_filename)
                    spearphishing_attachment_obj.add_attribute(object_relation="md5", simple_value=attachment_md5)
                    spearphishing_attachment_obj.add_attribute(object_relation="sha256", simple_value=attachment_sha256)
                    spearphishing_attachment_obj.add_attribute(object_relation="text",
                                                               simple_value=attachment_content_type, comment="content-type")

                    messages_delivered_event.add_object(spearphishing_attachment_obj)

            misp.add_event(messages_delivered_event)

    elif threat_type == "clicksBlocked":
        clicks_blocked_event = MISPEvent()
        clicks_blocked_event.info = info
        clicks_blocked_event.threat_level_id = 3  # low
        clicks_blocked_event.analysis = 2  # completed
        clicks_blocked_event.distribution = 1  # this community only

        for alert in tap_alerts[threat_type]:
            sender = alert['sender']
            sender_ip = alert['senderIP']
            blocked_clicked_url = alert['url']
            classification = alert['classification']

            # double check object and relations
            cb_object = MISPObject('spearphishing-link')
            cb_object.add_attribute(object_relation="email-sender", simple_value=sender)
            cb_object.add_attribute(object_relation="sender-ip", simple_value=sender_ip)
            cb_object.add_attribute(object_relation="embedded-link", simple_value=blocked_clicked_url)
            cb_object.comment = classification

            clicks_blocked_event.add_object(cb_object)
        misp.add_event(clicks_blocked_event)

    elif threat_type == "messagesBlocked":
        for alert in tap_alerts[threat_type]:
            messages_blocked_event = MISPEvent()
            messages_blocked_event.info = info
            messages_blocked_event.threat_level_id = 3  # low
            messages_blocked_event.analysis = 2  # completed
            messages_blocked_event.distribution = 1  # this community only

            sender = alert['sender']
            sender_header_from = alert['headerFrom']
            sender_header_reply_to = alert['headerReplyTo']
            sender_ip = alert['senderIP']
            subject = alert['subject']
            message_time = alert['messageTime']
            imposter_score = alert.get('imposterScore', 0)
            malware_score = alert.get('malwareScore', 0)
            phish_score = alert.get('phishScore', 0)
            spam_score = alert.get('spamScore', 0)

            messages_blocked_event.add_attribute(type="email-src", value=sender)
            messages_blocked_event.add_attribute(type="ip-src", value=sender_ip, category="Payload delivery")
            messages_blocked_event.add_attribute(type="email-subject", value=subject)

            if sender_header_from:
                messages_blocked_event.add_attribute(type="email-src-display-name", value=sender_header_from)
            if sender_header_reply_to:
                messages_blocked_event.add_attribute(type="email-reply-to", value=sender_header_reply_to)
            messages_blocked_event.add_attribute(type="datetime", value=message_time)

            if imposter_score > 0:
                messages_blocked_event.add_attribute(type="comment", value=imposter_score, comment="imposter score")
            if malware_score > 0:
                messages_blocked_event.add_attribute(type="comment", value=malware_score, comment="malware score")
            if phish_score > 0:
                messages_blocked_event.add_attribute(type="comment", value=phish_score, comment="phish score")
            if spam_score > 0:
                messages_blocked_event.add_attribute(type="comment", value=spam_score, comment="spam score")

            for threat in alert['threatsInfoMap']:
                classification = threat['classification']
                email_threat = threat['threat']
                threat_type = threat['threatType']

                if threat_type == "URL":
                    messages_blocked_event.add_attribute(type="domain", value=email_threat, category="Payload delivery",
                                                         comment=classification)
                elif threat_type == "ATTACHMENT":
                    messages_blocked_event.add_attribute(type="sha256", value=email_threat, categroy="Payload delivery",
                                                         comment=classification)

            for messagepart in alert['messageParts']:
                if messagepart['disposition'] == "attached":
                    attachment_filename = messagepart['filename']
                    attachment_md5 = messagepart['md5']
                    attachment_sha256 = messagepart['sha256']
                    attachment_content_type = messagepart['oContentType']

                    spearphishing_attachment_obj = MISPObject('file')
                    spearphishing_attachment_obj.comment = "malicious email attachment"
                    spearphishing_attachment_obj.add_attribute(object_relation="filename",
                                                               simple_value=attachment_filename)
                    spearphishing_attachment_obj.add_attribute(object_relation="md5", simple_value=attachment_md5)
                    spearphishing_attachment_obj.add_attribute(object_relation="sha256", simple_value=attachment_sha256)
                    spearphishing_attachment_obj.add_attribute(object_relation="text",
                                                               simple_value=attachment_content_type,
                                                               comment="content-type")

                    messages_blocked_event.add_object(spearphishing_attachment_obj)
            misp.add_event(messages_blocked_event)